<a href="https://colab.research.google.com/github/nikoardhiansyah/TugasPemrogramanKomputer/blob/main/Kelompok_11_(K)_Tugas_Minggu_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas geopandas rasterio numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 23.9 MB/s eta 0:00:00


In [2]:
import gdown
import geopandas as gpd
import pandas as pd
import rasterio

# ----------------------------------------
# 1. Google Drive File URLs & IDs
# ----------------------------------------

# GeoTIFF
tiff_url = "https://drive.google.com/uc?id=1-Zdq_ZXj4WoX5ubQnoHRjSlHfHrx07x0"
# GPKG (survey)
gpkg_url = "https://drive.google.com/uc?id=1AQcHlqzmpQIukLyPIkUnb04-pbjoET_y"
# Perioda CSV
periode_url = "https://drive.google.com/uc?id=1v8_2PwMxl1QGdcbC_RPgUGtYSZsmJDri"

# Output file names
tiff_file = "data_citra.tif"
gpkg_file = "data_survey.gpkg"
periode_file = "perioda.csv"

# ----------------------------------------
# 2. Download Files from Google Drive
# ----------------------------------------

print("Mengunduh data dari Google Drive...")
gdown.download(tiff_url, tiff_file, quiet=False)
gdown.download(gpkg_url, gpkg_file, quiet=False)
gdown.download(periode_url, periode_file, quiet=False)

# ----------------------------------------
# 3. Import Data
# ----------------------------------------

# Import GPKG
gdf = gpd.read_file(gpkg_file).to_crs(epsg=4326)
gdf["lon"] = gdf.geometry.x
gdf["lat"] = gdf.geometry.y
gdf["fid"] = gdf.index.astype(int)

# Rename tanggal → date jika perlu
if "tanggal" in gdf.columns:
    gdf = gdf.rename(columns={"tanggal": "date"})

# Pilih kolom yang diperlukan, pastikan kolom ada
required_cols = ['fid', 'lon', 'lat', 'date', 'fase']
missing_cols = [c for c in required_cols if c not in gdf.columns]
if missing_cols:
    raise ValueError(f"Kolom berikut tidak ditemukan di data survey: {missing_cols}")

output1 = gdf[required_cols].copy()
print("\n✅ Output 1: Filtered survey data")
print(output1.head())

# Import CSV periode
# Specify the separator as ';' because the columns are joined by semicolons
periode_df = pd.read_csv(periode_file, sep=';')
periode_df = periode_df.rename(columns={"Periode": "fase"})
periode_df['fase'] = periode_df['fase'].astype(str).str.strip()
output1['fase'] = output1['fase'].astype(str).str.strip()

# Merge survey dengan periode berdasarkan 'fase'
output2 = pd.merge(output1, periode_df, on="fase", how="left")

# Buat kolom 'periode' dari 'fase' sebagai int untuk band raster
# Ensure that the 'fase' column in output2 only contains values that can be converted to int
try:
    output2["periode"] = output2["fase"].astype(int)
except ValueError as e:
    print(f"Error converting 'fase' to int. Check for non-numeric values: {e}")
    # You might want to inspect the unique values in output2['fase'] here
    print("Unique values in output2['fase']:", output2['fase'].unique())
    raise # Re-raise the error after printing diagnostic info


print("\n✅ Output 2: Data survey + periode")
print(output2.head())

# Buka raster GeoTIFF
raster = rasterio.open(tiff_file)

# Fungsi ambil nilai pixel di koordinat lon, lat pada band tertentu
def get_pixel_value(lon, lat, band):
    try:
        # Check if band is a valid band index (1-based in rasterio)
        if not 1 <= band <= raster.count:
             # print(f"Warning: Band index {band} is out of valid range (1 to {raster.count}).")
             return None # Or handle as appropriate for your data

        # Sample the raster. rasterio.sample.sample_gen expects a list of points.
        # It returns a generator, so we iterate to get the value.
        for val in rasterio.sample.sample_gen(raster, [(lon, lat)], indexes=[band]):
            return val[0] # sample_gen returns an array of values, one per requested band
    except Exception as e:
        print(f"Error ambil pixel di ({lon}, {lat}) band {band}: {e}")
        return None

# Ekstrak nilai pixel citra sesuai periode (band)
# Apply the function to each row. The 'periode' column should now contain integer band indices.
output2["p0"] = output2.apply(lambda row: get_pixel_value(row["lon"], row["lat"], row["periode"]), axis=1)

# Output 3 dengan kolom lengkap
output3 = output2[['fid', 'lon', 'lat', 'date', 'fase', 'periode', 'p0']]
print("\n✅ Output 3: Data lengkap dengan nilai pixel")
print(output3.head())

# Simpan hasil jika ingin
output3.to_csv("output3_with_pixel.csv", index=False)
print("\n✅ Hasil disimpan di output3_with_pixel.csv")


Mengunduh data dari Google Drive...


Downloading...
From (original): https://drive.google.com/uc?id=1-Zdq_ZXj4WoX5ubQnoHRjSlHfHrx07x0
From (redirected): https://drive.google.com/uc?id=1-Zdq_ZXj4WoX5ubQnoHRjSlHfHrx07x0&confirm=t&uuid=cf274d19-8a23-453c-a1b9-debae7c02ade
To: /content/data_citra.tif
100%|██████████| 1.92G/1.92G [00:29<00:00, 64.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1AQcHlqzmpQIukLyPIkUnb04-pbjoET_y
To: /content/data_survey.gpkg
100%|██████████| 1.62M/1.62M [00:00<00:00, 142MB/s]
Downloading...
From: https://drive.google.com/uc?id=1v8_2PwMxl1QGdcbC_RPgUGtYSZsmJDri
To: /content/perioda.csv
100%|██████████| 827/827 [00:00<00:00, 2.41MB/s]



✅ Output 1: Filtered survey data
   fid         lon       lat       date  fase
0    0  107.463959 -6.260439  3/18/2024     1
1    1  107.464081 -6.260284  3/18/2024     1
2    2  107.463608 -6.260762  3/18/2024     1
3    3  107.461742 -6.261511  3/18/2024     1
4    4  107.463209 -6.260423  3/18/2024     1

✅ Output 2: Data survey + periode
   fid         lon       lat       date fase  Start Date    End Date  periode
0    0  107.463959 -6.260439  3/18/2024    1  2024-01-01  2024-01-13        1
1    1  107.464081 -6.260284  3/18/2024    1  2024-01-01  2024-01-13        1
2    2  107.463608 -6.260762  3/18/2024    1  2024-01-01  2024-01-13        1
3    3  107.461742 -6.261511  3/18/2024    1  2024-01-01  2024-01-13        1
4    4  107.463209 -6.260423  3/18/2024    1  2024-01-01  2024-01-13        1

✅ Output 3: Data lengkap dengan nilai pixel
   fid         lon       lat       date fase  periode    p0
0    0  107.463959 -6.260439  3/18/2024    1        1 -1379
1    1  107.464081 -6.

In [3]:
import gdown
import geopandas as gpd
import pandas as pd
import rasterio

# ----------------------------------------
# 1. Google Drive File URLs & IDs
# ----------------------------------------

# GeoTIFF
tiff_url = "https://drive.google.com/uc?id=1-Zdq_ZXj4WoX5ubQnoHRjSlHfHrx07x0"
# GPKG (survey)
gpkg_url = "https://drive.google.com/uc?id=1AQcHlqzmpQIukLyPIkUnb04-pbjoET_y"
# Perioda CSV
periode_url = "https://drive.google.com/uc?id=1v8_2PwMxl1QGdcbC_RPgUGtYSZsmJDri"

# Output file names
tiff_file = "data_citra.tif"
gpkg_file = "data_survey.gpkg"
periode_file = "perioda.csv"

# ----------------------------------------
# 2. Download Files from Google Drive
# ----------------------------------------

print("Mengunduh data dari Google Drive...")
gdown.download(tiff_url, tiff_file, quiet=False)
gdown.download(gpkg_url, gpkg_file, quiet=False)
gdown.download(periode_url, periode_file, quiet=False)

# ----------------------------------------
# 3. Import Data
# ----------------------------------------

# Import GPKG dan ubah koordinat ke EPSG:4326
gdf = gpd.read_file(gpkg_file).to_crs(epsg=4326)
gdf["lon"] = gdf.geometry.x
gdf["lat"] = gdf.geometry.y
gdf["fid"] = gdf.index.astype(int)

# Ganti nama kolom 'tanggal' menjadi 'date' jika ada
if "tanggal" in gdf.columns:
    gdf = gdf.rename(columns={"tanggal": "date"})

# Pilih kolom yang diperlukan
required_cols = ['fid', 'lon', 'lat', 'date', 'fase']
missing_cols = [c for c in required_cols if c not in gdf.columns]
if missing_cols:
    raise ValueError(f"Kolom berikut tidak ditemukan di data survey: {missing_cols}")

output1 = gdf[required_cols].copy()
print("\n✅ Output 1: Filtered survey data")
print(output1.head())

# Import CSV periode
# Karena pemisahnya adalah ';'
periode_df = pd.read_csv(periode_file, sep=';')
periode_df = periode_df.rename(columns={"Periode": "fase"})
periode_df['fase'] = periode_df['fase'].astype(str).str.strip()
output1['fase'] = output1['fase'].astype(str).str.strip()

# Merge survey dengan periode berdasarkan 'fase'
output2 = pd.merge(output1, periode_df, on="fase", how="left")

# Buat kolom 'periode' dari 'fase' sebagai int untuk band raster
try:
    output2["periode"] = output2["fase"].astype(int)
except ValueError as e:
    print(f"Error converting 'fase' to int. Check for non-numeric values: {e}")
    print("Unique values in output2['fase']:", output2['fase'].unique())
    raise

print("\n✅ Output 2: Data survey + periode")
print(output2.head())

# Buka raster GeoTIFF
raster = rasterio.open(tiff_file)

# Fungsi untuk mengambil nilai pixel dari koordinat (lon, lat) pada band tertentu.
def get_pixel_value(lon, lat, band):
    try:
        # Pastikan band dalam rentang valid (1-based index)
        if not 1 <= band <= raster.count:
            return None
        # Menggunakan rasterio.sample.sample_gen untuk mengambil sampel pixel
        for val in rasterio.sample.sample_gen(raster, [(lon, lat)], indexes=[band]):
            return val[0]
    except Exception as e:
        print(f"Error ambil pixel di ({lon}, {lat}) band {band}: {e}")
        return None

# Fungsi untuk mengambil nilai pixel dengan offset periode
# p0 = output2["periode"], p1 = output2["periode"] - 1, dst.
def get_pixel_value_offset(row, offset):
    target_band = row["periode"] - offset
    # Jika target_band tidak valid, kembalikan None
    if target_band < 1 or target_band > raster.count:
        return None
    return get_pixel_value(row["lon"], row["lat"], target_band)

# Ekstrak nilai pixel untuk p0 hingga p6
for offset in range(7):
    col_name = f"p{offset}"
    output2[col_name] = output2.apply(lambda row: get_pixel_value_offset(row, offset), axis=1)

# Pilih kolom final untuk output
output3 = output2[['lon', 'lat', 'date', 'fase', 'periode', 'p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6']]
print("\n✅ Output 3: Data lengkap dengan nilai pixel untuk p0 hingga p6")
print(output3.head())

# Simpan hasil ke file CSV
output3.to_csv("output3_with_pixels.csv", index=False)
print("\n✅ Hasil disimpan di output3_with_pixels.csv")


Mengunduh data dari Google Drive...


Downloading...
From (original): https://drive.google.com/uc?id=1-Zdq_ZXj4WoX5ubQnoHRjSlHfHrx07x0
From (redirected): https://drive.google.com/uc?id=1-Zdq_ZXj4WoX5ubQnoHRjSlHfHrx07x0&confirm=t&uuid=10a79901-4b0e-4c44-aca6-0296753f78c1
To: /content/data_citra.tif
100%|██████████| 1.92G/1.92G [00:17<00:00, 111MB/s]
Downloading...
From: https://drive.google.com/uc?id=1AQcHlqzmpQIukLyPIkUnb04-pbjoET_y
To: /content/data_survey.gpkg
100%|██████████| 1.62M/1.62M [00:00<00:00, 112MB/s]
Downloading...
From: https://drive.google.com/uc?id=1v8_2PwMxl1QGdcbC_RPgUGtYSZsmJDri
To: /content/perioda.csv
100%|██████████| 827/827 [00:00<00:00, 1.65MB/s]



✅ Output 1: Filtered survey data
   fid         lon       lat       date  fase
0    0  107.463959 -6.260439  3/18/2024     1
1    1  107.464081 -6.260284  3/18/2024     1
2    2  107.463608 -6.260762  3/18/2024     1
3    3  107.461742 -6.261511  3/18/2024     1
4    4  107.463209 -6.260423  3/18/2024     1

✅ Output 2: Data survey + periode
   fid         lon       lat       date fase  Start Date    End Date  periode
0    0  107.463959 -6.260439  3/18/2024    1  2024-01-01  2024-01-13        1
1    1  107.464081 -6.260284  3/18/2024    1  2024-01-01  2024-01-13        1
2    2  107.463608 -6.260762  3/18/2024    1  2024-01-01  2024-01-13        1
3    3  107.461742 -6.261511  3/18/2024    1  2024-01-01  2024-01-13        1
4    4  107.463209 -6.260423  3/18/2024    1  2024-01-01  2024-01-13        1

✅ Output 3: Data lengkap dengan nilai pixel untuk p0 hingga p6
          lon       lat       date fase  periode    p0  p1  p2  p3  p4  p5  \
0  107.463959 -6.260439  3/18/2024    1      